In [2]:
#!pip install yfinance --upgrade --force-reinstall

In [1]:
import yfinance as yf
import pandas as pd
import time

# List of stock tickers (Example)
tickers = ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN"]
sp100 = [
    "AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "GOOG", "META", "TSLA", "BRK-B", "UNH",
    "JPM", "JNJ", "XOM", "V", "PG", "MA", "LLY", "HD", "AVGO", "CVX",
    "ABBV", "COST", "MRK", "PEP", "KO", "MCD", "ADBE", "TMO", "NFLX", "CRM",
    "AMD", "PFE", "ABT", "INTC", "CMCSA", "ACN", "LIN", "VZ", "ORCL", "WFC",
    "DIS", "TXN", "NKE", "QCOM", "DHR", "PM", "RTX", "IBM", "UNP", "CAT",
    "LOW", "GS", "HON", "SPGI", "NOW", "MS", "BLK", "SCHW", "LMT", "BA",
    "MDT", "ELV", "BKNG", "T", "ISRG", "GILD", "GE", "AMAT", "DE", "C",
    "PLD", "DUK", "BSX", "PYPL", "SBUX", "AXP", "TGT", "MO", "CB", "ADP",
    "SO", "CI", "AMGN", "EQIX", "BDX", "USB", "REGN", "MMC", "ADI", "MU",
    "PGR", "VRTX", "MDLZ", "ZTS", "PNC", "CME", "CSX", "TFC", "EOG", "ICE"
]

etfs = ["EWJ","EWG","EWU","INDA","IEV","MCHI","XLE","REZ","IFY","IYW","IYC","IYH","IYZ","IYJ","IYK"]

tickers = sp100

# Create an empty DataFrame
df = pd.DataFrame(columns=["Ticker", "Company Name", "PE Ratio TTM", "PE Forward", "ROE", "ROA", "Dividend Paid"])

# Temporary list to store data
data_list = []

# Fetch financial data for each ticker
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        print(f"Fetching data for: {ticker}")
        time.sleep(0.10)  # Sleep to avoid rate limits

        # Extract relevant data, handling missing values
        hist = stock.history(period="1d")
        info = stock.info
        pe_ratio_ttm = info.get("trailingPE", None)
        pe_forward = info.get("forwardPE", None)
        roe = info.get("returnOnEquity", None)
        roa = info.get("returnOnAssets", None)
        dividend_paid = info.get("dividendRate", None)
        company_name = info.get("shortName", ticker)  # Use ticker if name is unavailable
        beta = info.get("beta", None)  # Stock beta
        closing_price = hist["Close"].iloc[-1] if not hist.empty else None  # Most recent closing price
        one_year_target = info.get("targetMeanPrice", None)


        # Append data to list
        data_list.append({
            "Ticker": ticker,
            "Company Name": company_name,
            "Price": closing_price,
            "Beta": beta,
            "PE Ratio TTM": pe_ratio_ttm,
            "PE Forward": pe_forward,
            "ROE": roe,
            "ROA": roa,
            "Dividend Paid": dividend_paid,
            "YTarget":one_year_target
        })

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        time.sleep(5)  # Wait if error occurs


        
# Convert list to DataFrame using pd.concat()
df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)

# Save to CSV
df.to_csv("financial_data.csv", index=False)
print("Data saved to financial_data.csv")

Fetching data for: AAPL
Fetching data for: MSFT
Fetching data for: AMZN
Fetching data for: NVDA
Fetching data for: GOOGL
Fetching data for: GOOG
Fetching data for: META
Fetching data for: TSLA
Fetching data for: BRK-B
Fetching data for: UNH
Fetching data for: JPM
Fetching data for: JNJ
Fetching data for: XOM
Fetching data for: V
Fetching data for: PG
Fetching data for: MA
Fetching data for: LLY
Fetching data for: HD
Fetching data for: AVGO
Fetching data for: CVX
Fetching data for: ABBV
Fetching data for: COST
Fetching data for: MRK
Fetching data for: PEP
Fetching data for: KO
Fetching data for: MCD
Fetching data for: ADBE
Fetching data for: TMO
Fetching data for: NFLX
Fetching data for: CRM
Fetching data for: AMD
Fetching data for: PFE
Fetching data for: ABT
Fetching data for: INTC
Fetching data for: CMCSA
Fetching data for: ACN
Fetching data for: LIN
Fetching data for: VZ
Fetching data for: ORCL
Fetching data for: WFC
Fetching data for: DIS
Fetching data for: TXN
Fetching data for: NK

/var/folders/0d/kg6h6hgn2rj5smljr1pqgk0r0000gn/T/ipykernel_23399/552118445.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)


In [2]:
df["PE_Rank"] = df["PE Ratio TTM"].rank(method="min") 
# df["PE_Score"] = ((df["PE Ratio TTM"].max() - df["PE Ratio TTM"]) / (df["PE Ratio TTM"].max() - df["PE Ratio TTM"].min())) * 4 + 1
df["PE_Score"] = 4* ((df["PE_Rank"].max() - df["PE_Rank"]) / (df["PE_Rank"].max() - df["PE_Rank"].min())) + 1

# Scale PE Forward (Lower is Better, so Inverted)
df["PE_Forward_Score"] = 5 - (4*(( df["PE Forward"]-df["PE Forward"].min()) / 
                          (df["PE Forward"].max() - df["PE Forward"].min())))

# Scale ROE (Higher is Better)
df["ROER"] = df["ROE"].rank(method="max") 
df["ROE_Score"] = ((df["ROER"] - df["ROER"].min()) / 
                   (df["ROER"].max() - df["ROER"].min())) * 4 + 1

# Scale ROA (Higher is Better)
df["ROAR"] = df["ROA"].rank(method="max") 
df["ROA_Score"] = ((df["ROAR"] - df["ROAR"].min()) / 
                   (df["ROAR"].max() - df["ROAR"].min())) * 4 + 1

# Scale Dividend Paid (Higher is Better)
df["DivR"] = df["Dividend Paid"].rank(method="max") 
df["Dividend_Score"] = ((df["DivR"] - df["DivR"].min()) / 
                        (df["DivR"].max() - df["DivR"].min())) * 4 + 1

#df["PE_Score"] = df.apply(lambda x: 1.0 if x["PE_Score"] < df["PE_Score"].median() else x["PE_Score"], axis=1)

df["Score"] = (0.35 * df["PE_Score"] + 
               0.35 * df["ROE_Score"] + 
               0.15 * df["ROA_Score"] + 
               0.15 * df["Dividend_Score"])

threshold = df["PE Ratio TTM"].quantile(0.6)  # Find the 60th percentile (top 40% starts above this)
df["Score"] = df.apply(lambda x: 0 if x["PE Ratio TTM"] > threshold else x["Score"], axis=1)


ordered_columns = ["Ticker", "Company Name", "Price", "YTarget", "Beta", "Score", "PE_Score","PE Ratio TTM", "PE Forward", "ROE_Score", "ROE", "ROA_Score","ROA", "Dividend_Score", "Dividend Paid"]
# Reorder DataFrame
df = df[ordered_columns]

df = df.sort_values(by="Score", ascending=False)
# Save to CSV
df.to_csv("financial_data_35_35_15_15_all_final.csv", index=False)
print("Data saved to financial_data.csv")


Data saved to financial_data.csv


In [3]:
df = df.sort_values(by="Score", ascending=False)
# Save to CSV
df.to_csv("financial_data_25_all_final.csv", index=False)
print("Data saved to financial_data.csv")


Data saved to financial_data.csv


In [4]:
df

,Ticker,Company Name,Price,YTarget,Beta,Score,PE_Score,PE Ratio TTM,PE Forward,ROE_Score,ROE,ROA_Score,ROA,Dividend_Score,Dividend Paid
49,CAT,"Caterpillar, Inc.",329.690002,391.12564,1.138,4.312249,4.368421,14.945150,14.884425,4.565217,0.55327,3.868687,0.09675,4.034483,5.64
17,HD,"Home Depot, Inc. (The)",358.149994,431.49567,1.090,4.266110,3.231579,24.036913,22.958332,5.000000,3.85372,4.555556,0.15585,4.678161,9.20
58,LMT,Lockheed Martin Corporation,441.489990,523.76044,0.427,4.183028,3.652632,19.797758,15.705798,4.695652,0.81045,3.545455,0.08306,4.862069,13.20
22,MRK,"Merck & Company, Inc.",89.230003,112.06682,0.363,4.129562,4.494737,13.238874,9.492554,4.260870,0.40789,4.353535,0.13903,2.747126,3.24
43,QCOM,QUALCOMM Incorporated,152.720001,198.49700,1.289,4.038904,4.200000,16.474650,12.487327,4.304348,0.41968,4.151515,0.12626,2.931034,3.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,BKNG,Booking Holdings Inc. Common St,4634.240234,5636.97600,1.404,NaN,2.852632,26.809212,22.083584,NaN,NaN,4.838384,0.18225,5.000000,38.40
73,PYPL,"PayPal Holdings, Inc.",65.150002,93.60400,1.527,NaN,4.242105,16.328321,13.323109,3.130435,0.20001,2.252525,0.04299,NaN,NaN
77,MO,"Altria Group, Inc.",58.150002,56.58333,0.606,NaN,4.957895,8.891438,10.869160,NaN,NaN,4.878788,0.20511,3.436782,4.08
91,VRTX,Vertex Pharmaceuticals Incorpor,492.690002,497.01172,0.415,NaN,NaN,NaN,26.248800,1.043478,-0.03152,4.191919,0.12683,NaN,NaN
